In [25]:
import pickle
import glob
import os
import rasterio
import numpy as np
from scipy.ndimage import median_filter
import pandas as pd

In [26]:
#data_dir = ""
out_dir = f"C:/Users/apsam/Documents/work/Thiyaku work/blog_urban_growth/classified_v2"

model_path = f"C:/Users/apsam/Documents/work/Thiyaku work/blog_urban_growth/random_forest_model_cbe.pkl"

In [27]:
with open(model_path, "rb") as f:
    model = pickle.load(f)

In [28]:
tif_paths = glob.glob(f"C:/Users/apsam/Documents/work/Thiyaku work/blog_urban_growth/New folder/*.tif")

In [29]:
def predict_tif(input_path, output_path):
    # Open the .tif file
    with rasterio.open(input_path) as src:
        # Read all bands into a 3D array (height x width x bands)
        image = np.stack([src.read(i) for i in range(1, src.count + 1)], axis=-1)
        
        # Reshape the image to have one row per pixel (num_pixels x bands)
        nrows, ncols, nbands = image.shape
        reshaped_image = image.reshape((nrows * ncols, nbands))
        
        # Define the expected bands (assuming the model expects "B1", "B2", ..., "B13")
        expected_bands = 13  # Change this to match the number of bands used during training
        if nbands > expected_bands:
            reshaped_image = reshaped_image[:, :expected_bands]  # Select only the first expected_bands columns
        
        # Convert to DataFrame with appropriate column names for the model
        band_columns = [f"B{i}" for i in range(1, expected_bands + 1)]
        X_predict = pd.DataFrame(reshaped_image, columns=band_columns)
        
        # Predict the class for each pixel
        predictions = model.predict(X_predict)
        
        # Reshape predictions back to the original image shape
        classified_image = predictions.reshape((nrows, ncols))
    
    # Filter the predictions (assuming class 2 corresponds to settlements)
    #filtered_image = np.where(classified_image == 2, 1, 0)
    filtered_image = median_filter(classified_image, size=3)
    
    # Save the classified image as a new .tif file
    output_meta = src.meta.copy()
    output_meta.update({
        "driver": "GTiff",
        "height": nrows,
        "width": ncols,
        "count": 1,
        "dtype": "int32",  # assuming the classes are integers
        "transform": src.transform
    })
    with rasterio.open(output_path, "w", **output_meta) as dst:
        dst.write(filtered_image.astype(rasterio.int32), 1)
    
    return True

In [30]:
for tif_path in tif_paths:
    tif_bn = os.path.basename(tif_path)
    tif_date = tif_bn.replace(".tif", "")
    out_path = os.path.join(out_dir, tif_bn)
    if not os.path.exists(out_path):
        predict_tif(tif_path, out_path)
    print(out_path)

C:/Users/apsam/Documents/work/Thiyaku work/blog_urban_growth/classified_v2\20230222T050821_20230222T052224_T43PGN.tif
C:/Users/apsam/Documents/work/Thiyaku work/blog_urban_growth/classified_v2\20230227T050749_20230227T052458_T43PGN.tif
C:/Users/apsam/Documents/work/Thiyaku work/blog_urban_growth/classified_v2\20240207T051001_20240207T052026_T43PGN.tif
